In [ ]:
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.datasets import imdb

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
endpoint_name = '<your-endpoint>'

class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer, 
                         deserializer=json_deserializer, content_type='application/json')

sentiment_predictor = Predictor(endpoint_name, sagemaker_session=sagemaker_session)

In [ ]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
regex = re.compile(r'^[\?\s]+')
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
review_index = 10
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_test[review_index]])
print(regex.sub('', decoded_review))

In [ ]:
results = sentiment_predictor.predict(x_test[review_index])['predictions'][0][0]

def get_sentiment(score):
    return 'positive' if score > 0.5 else 'negative' 

print('Labeled sentiment for this review is {}, predicted sentiment is {}'.format(get_sentiment(y_test[review_index]), 
                                                                                  get_sentiment(results)))